In [18]:
%pip install pandas -q
%pip install numpy -q

import numpy as np
import pandas as pd
import random
from collections import Counter
import copy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## Data Preparation

In [19]:
# data = [job0, job1, .... ]
# job1 = , [op0, op1, op2, ....]
# op1 = [{macineNumber: , processTime: },{machineNumber: , processTime: }...]

with open ("../Dataset/abz5.fjs","r") as file:
    firstLine = file.readline()
    firstLineValues = [int(i) for i in firstLine.split()[0:2]]
    NUM_JOBS = firstLineValues[0] 
    NUM_MACHINES = firstLineValues[1]
    print(f"numJobs = {NUM_JOBS}, numMachines = {NUM_MACHINES}")
    
    data = []  
    for txts in file.readlines():
        jobNum = []
        operations = txts.split()[1:]
        index = 0
        while index < len(operations):
            operation = []
            numberOfOperation = int(operations[index])
            index += 1
            endOperationIndex = numberOfOperation*2 + index
            while index < endOperationIndex:
                operation.append({"machineNumber": int(operations[index]), "processTime": int(operations[index+1])})
                index += 2
            jobNum.append(operation)
        data.append(jobNum)
        
print(f"number of first job's operation = {len(data[0])}")
print(f"number of machine that first job's operation can use = {len(data[0][0])}")

numJobs = 10, numMachines = 10
number of first job's operation = 10
number of machine that first job's operation can use = 5


In [20]:
# set we have total n orders
# order_i (i from 1~n) = [{product: 1, qunatity: x}, ..., {prodcut: n, quantity: x}]
# demands = [order_1, ..., order_n]
# default 3 demands, note: product_0 can only be produced by completing job_0 
order_1 = [{"product":0, "quantity":2}, {"product":1, "quantity":2}, {"product":2, "quantity":2}]
order_2 = [{"product":5, "quantity":2}]
order_3 = [{"product":0, "quantity":1}, {"product":1, "quantity":1}, {"product":2, "quantity":1}, {"product":3, "quantity":1}, {"product":4, "quantity":1}, {"product":5, "quantity":1}, {"product":6, "quantity":1}, {"product":7, "quantity":1}, {"product":8, "quantity":1}, {"product":9, "quantity":1}]
demands = [order_1, order_2, order_3]

# dueDate = [i for i in (dueDate for each order)]
dueDate = [np.inf, np.inf, np.inf]

## Hyper Parameter

In [21]:
POP_SIZE = 30
CROSSOVER_RATE = 0.7
MAX_ITERATION = 400
NUMBER_OF_ELITISM_SURVIVORS = 2
NUMBER_OF_TOURNAMENT_COMPETITORS = 2

## Generate Initial Population

In [22]:
# Initial population = [[J0, M3], [J0, M4], [J2, M1], [J3, M2], [J0, M1], ...]
# meaning J1 first operation use Machine 3 -> J2 first operation use Machine 4  -> J1 second operation......

def generateInitailPopulation():
    # create all jobs with specific number of operations
    # because different Jobs could have different number of operations
    baseArray = []
    for orders in demands:   
        for order in orders:
            quantity = order["quantity"]
            for _ in range(quantity):
                baseArray.append(order["product"])
    
    population = []
    # random choose that jobs operation for
    for _ in range(POP_SIZE):
        # how many operations have done
        operationCounter = Counter()
        individual = []
        baseArray = np.random.permutation(baseArray)
        for jobNum in baseArray:
            machine = random.choice(data[jobNum][operationCounter[jobNum]])
            operationCounter[jobNum]+=1
            individual.append([jobNum, machine["machineNumber"]])
            # if one job is finished (all operation has been done), then restart counting
            if operationCounter[jobNum] == len(data[jobNum]):
                operationCounter[jobNum] = 0
        population.append(individual)
    return population

## Repairment

In [23]:
# repairment deal with infieasible solution (the operation can't performed by that machine)
# When happens, it randomly choose a feasilbe machine that can perform that operation
def repair(unrepaired):
    operationCounter = Counter()
    repairedList = copy.deepcopy(unrepaired)
    for i in range(len(unrepaired)):
        jobNum = unrepaired[i][0]
        machineNum = unrepaired[i][1]
        machines = [i["machineNumber"] for i in data[jobNum][operationCounter[jobNum]]]
        if machineNum not in machines:
            repairedList[i][1] = random.choice(machines)
        operationCounter[jobNum]+=1
        # if one job is finished (all operation has been done), then restart counting
        if operationCounter[jobNum] == len(data[jobNum]):
            operationCounter[jobNum] = 0
    return repairedList

## Fitness Evaluation

In [24]:
# note: when a job finished, a product will be deleiver to an ordered that is the most urgent(valuable)
def evaluateFitnessScore(chromosome):
    
    # OperationCounter shows how many times of operation have been done
    # jobEndCounter shows the time that last Ji ends
    # machineTimeCounter shows the time that last job ends
    # sortedDemands shows the number of jobs rest to be done in required order (urgency)
    operationCounter = Counter()
    jobEndCounter = Counter()
    machineTimeCounter = Counter()
    sortedDemands = [(demands[i],dueDate[i]) for i in sorted(range(len(dueDate)), key=lambda x: dueDate[x])]

    tardiness = 0
    for jobIteration in chromosome:
        jobNum = jobIteration[0]
        machineNumber = jobIteration[1]
        
        # calculate the duration of that specific operation
        operationsDictList = data[jobNum][operationCounter[jobNum]]
        duration = [i["processTime"] for i in operationsDictList if i["machineNumber"] == machineNumber][0]
         
        # update the counter
        machineTimeCounter[machineNumber] = max(machineTimeCounter[machineNumber], jobEndCounter[jobNum])
        machineTimeCounter[machineNumber] += duration
        jobEndCounter[jobNum] = machineTimeCounter[machineNumber]
        operationCounter[jobNum] += 1
        
        # if one job is finished (all operation has been done), then restart counting
        if operationCounter[jobNum] == len(data[jobNum]):
            operationCounter[jobNum] = 0
            for index in range(len(sortedDemands)):
                for order in sortedDemands[index][0]:
                    if order["product"] == jobNum and order["quantity"]!=0 :
                        order["quantity"] -= 1
                        break
                    
            # calcualte tardiness
            for index in range(len(sortedDemands)):
                done = 1
                for order in sortedDemands[index][0]:
                    if order["quantity"] != 0:
                        done = 0
                if done == 1:
                    tardiness += max(0, jobEndCounter[jobNum] - sortedDemands[index][1])
                    del sortedDemands[index]
                    
                
    # calcualte makespan 
    makespan = jobEndCounter.most_common(1)[0][1]
    
    return makespan + tardiness


## Selection

In [25]:
# Tournament method
def selection(populatoinWithMakespan):
    population = populatoinWithMakespan["population"]
    makespan = populatoinWithMakespan["makespan"]
    selected = []
    for _ in range(2):
        competitors = np.random.permutation(len(population))[:NUMBER_OF_TOURNAMENT_COMPETITORS]
        
        v = max(makespan[i] for i in competitors)
        for i in competitors:
            if makespan[i]==v:
                selected.append([population[i], makespan[i]])
                break
        
    return pd.DataFrame(selected,columns=["population", "makespan"])

##  Crossover & Mutation

In [26]:
# Davis’ Order Crossover 
def crossover(individual1, individual2):
    length = len(individual1)

    # Initialize the offspring with None values
    offspring1 = [None] * length
    offspring2 = [None] * length
    
    # Choose two crossover points
    cxpoint1, cxpoint2 = sorted(random.sample(range(length), 2))
    
    # Copy the sub-sequence from parent1 to offspring1 and parent2 to offspring2
    offspring1[cxpoint1:cxpoint2] = individual1[cxpoint1:cxpoint2]
    offspring2[cxpoint1:cxpoint2] = individual2[cxpoint1:cxpoint2]
    
    # Create a counter for each offspring
    counter1 = Counter(i[0] for i in individual1) - Counter(i[0] for i in offspring2[cxpoint1:cxpoint2]) 
    counter2 = Counter(i[0] for i in individual2) - Counter(i[0] for i in offspring1[cxpoint1:cxpoint2])
    
    def fill_offspring(parent, offspring, counter):
        current_pos = 0
        for gene in parent:
            jobNum = gene[0]
            machineNum = gene[1]
            if counter[jobNum] > 0:
                while offspring[current_pos] is not None:
                    current_pos = (current_pos + 1) % length
                offspring[current_pos] = [jobNum, machineNum]  
                counter[jobNum] -= 1
        # If there are still None values left, fill them with remaining genes
        for i in range(length):
            if offspring[i] is None:
                offspring[i] = parent[current_pos]
                current_pos = (current_pos + 1) % length
    
    # Fill the rest of the genes in the offspring
    fill_offspring(individual2, offspring1, counter1)
    fill_offspring(individual1, offspring2, counter2)
    
    return [repair(offspring1), repair(offspring2)]

# Change side if 
def mutate(individual):
    mutateOffspring = copy.deepcopy(individual)
    i, j = random.sample(range(len(individual)), 2)
    mutateOffspring[i], mutateOffspring[j] = mutateOffspring[j], mutateOffspring[i]
    return repair(mutateOffspring)

## Elitism Strategy

In [27]:
def elitismStrategy(population):
    sorted_df = population.sort_values(by=["makespan"], ascending=[True])
    survivors_df = sorted_df.head(NUMBER_OF_ELITISM_SURVIVORS)
    return survivors_df

## Main Function

In [28]:

iteration = 0
populationList = []
lineGraphDataList = []
elitismIndividualDF = pd.DataFrame()

while(iteration <= MAX_ITERATION):
    if len(populationList) == 0 :
        populationList = generateInitailPopulation()
    else:
        populationList = [i[0] for i in offspringList]
        
    # scored the population
    scoredPopulationList = []
    for chromosome in populationList:
        makespan = evaluateFitnessScore(chromosome)
        scoredPopulationList.append([chromosome, makespan])
    
    # select two individual from the population and crossover & mutation until meet the pop size
    offspringList = []
    scoredPopulationList.extend(elitismIndividualDF.values)
    while len(offspringList) <= POP_SIZE:
        selectedIndividual = selection(pd.DataFrame(scoredPopulationList,columns=["population", "makespan"]))
        if random.random() < CROSSOVER_RATE:
            offsprings = crossover(selectedIndividual.iloc[0,0],selectedIndividual.iloc[1,0])
            for offspring in offsprings:
                makespan = evaluateFitnessScore(offspring)
                offspringList.append([offspring, makespan])
        else:
            offspring = mutate(selectedIndividual.iloc[0,0])
            makespan = evaluateFitnessScore(offspring)
            offspringList.append([offspring, makespan])

    
    # sort by makespan (ascending)
    offspringList.extend(elitismIndividualDF.values)
    elitismIndividualDF = elitismStrategy(pd.DataFrame(offspringList, columns=["population", "makespan"]))
    
    iteration+=1
    lineGraphDataList.append(elitismIndividualDF.iloc[0,1])
    print(f"interation: {iteration}")
    print(f"best makespan: {elitismIndividualDF.iloc[0,1]} \n")

interation: 1
best makespan: 315 



KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(len(lineGraphDataList)), lineGraphDataList, label='makespan')
plt.xlabel('Period')
plt.ylabel('Value')
plt.title('Trend of Makespan')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
def evaluateFitnessScore2(chromosome):
    
    # Job shows how many times of operation have been done
    # Job_end shows the time that last Ji ends
    # machine time shows the time that last job ends
    jobCounter = Counter()
    jobEndCounter = Counter()
    machineTimeCounter = Counter()
    
    for jobIteration in chromosome:
        jobNumber = jobIteration[0]
        machineNumber = jobIteration[1]
        
        # calculate the duration of that specific operation
        operationsDictList = data[jobNumber][jobCounter[jobNumber]]
        duration = [i["processTime"] for i in operationsDictList if i["machineNumber"] == machineNumber][0]
         
        # update the counter
        machineTimeCounter[machineNumber] = max(machineTimeCounter[machineNumber], jobEndCounter[jobNumber])
        machineTimeCounter[machineNumber] += duration
        jobEndCounter[jobNumber] = machineTimeCounter[machineNumber]
        jobCounter[jobNumber] += 1
        
    # calcualte makespan & tardiness
    makespan = jobEndCounter.most_common(1)[0][1]
    
    return makespan, jobEndCounter

In [ ]:
print(elitismIndividualDF.iloc[0])
score, counter = evaluateFitnessScore2(elitismIndividualDF.iloc[0, 0])
print(counter)
counter.keys()